# Google gmaps

## 6.5.2 Create Heatmaps for Weather Parameters

In [1]:
# Import dependencies
import pandas as pd
import gmaps
import requests

# import api key
from config import g_key

# import widgets
import ipywidgets as widgets

In [2]:
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Petropavl,KZ,2021-09-22 19:03:27,54.8753,69.1628,36.01,93,39,6.80
1,1,Mayo,US,2021-09-22 19:03:27,38.8876,-76.5119,83.17,77,20,15.99
2,2,Guerrero Negro,MX,2021-09-22 18:58:39,27.9769,-114.0611,82.06,58,100,6.24
3,3,Busselton,AU,2021-09-22 19:03:27,-33.6500,115.3333,51.04,91,0,10.07
4,4,Dickinson,US,2021-09-22 19:03:28,29.4608,-95.0513,83.23,43,1,4.00


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

### Create a Maximum Temperature Heatmap

In [4]:
# Configure gmaps to use your Google API key
gmaps.configure(api_key=g_key)

In [54]:
def create_heat_map(locations, weights, marker_info=None):
    """
    create new heat map with gmaps for locations and weights passed in
    
    args:
        locations: DataFrame with two columns [["Lat", "Lng"]]
        weights: List of max temperatures or humidity, or another weather metric
    """ 
    # create figure
    fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
    
    heat_layer = gmaps.heatmap_layer(locations, weights, dissipating=False, max_intensity=300, point_radius=4)
        
    # add the heatmap layers
    fig.add_layer(heat_layer)

    # add marker layer if marker_info argument is provided
    if (marker_info != None):
        marker_layer = gmaps.marker_layer(locations, info_box_content=marker_info)
        fig.add_layer(marker_layer)

    
    # call figure to plot the data
    return fig

In [6]:
# Get the maximum temperature
max_temp = city_data_df["Max Temp"]
temps = [max(temp,0) for temp in max_temp]

In [15]:
# Heatmap of temperature
# Get latitude and longitude
locations = city_data_df[["Lat", "Lng"]]

# Get the maximum temperature
max_temp = city_data_df["Max Temp"]

# Assign the figure variable
fig = gmaps.figure()

# Assign the heatmap variable
# Adjust Heatmap Zoom, Intensity, and Point Radius
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp,0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)

# Add the heatmap layer
fig.add_layer(heat_layer)

# call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

### Create Percent Humidity Heatmap

In [17]:
# Heatmap of percent hunidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create Wind Speed Heatmap

In [55]:
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]

fig = create_heat_map(locations, wind)
fig

Figure(layout=FigureLayout(height='420px'))

## 6.5.3 Get Vacation Criteria

In [32]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 72
What is the maximum temperature you would like for your trip? 85


In [33]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Mayo,US,2021-09-22 19:03:27,38.8876,-76.5119,83.17,77,20,15.99
2,2,Guerrero Negro,MX,2021-09-22 18:58:39,27.9769,-114.0611,82.06,58,100,6.24
4,4,Dickinson,US,2021-09-22 19:03:28,29.4608,-95.0513,83.23,43,1,4.00
5,5,Alugan,PH,2021-09-22 19:03:28,12.2188,125.4808,78.03,85,100,2.10
6,6,Rikitea,PF,2021-09-22 19:03:28,-23.1203,-134.9692,73.27,75,19,20.76
7,7,Bambous Virieux,MU,2021-09-22 19:03:29,-20.3428,57.7575,72.21,72,15,13.91
11,11,Katsuura,JP,2021-09-22 19:03:30,35.1333,140.3000,80.71,85,100,19.55
16,16,Campoverde,PE,2021-09-22 19:03:31,-8.4753,-74.8058,73.09,49,88,5.99
18,18,Bethel,US,2021-09-22 19:03:31,41.3712,-73.4140,80.35,75,1,1.99
22,22,Wanning,CN,2021-09-22 19:03:32,18.8003,110.3967,80.04,87,100,4.21


In [34]:
preferred_cities_df.isnull().sum()

City_ID       0
City          0
Country       1
Date          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
dtype: int64

In [35]:
preferred_cities_df.count()

City_ID       205
City          205
Country       204
Date          205
Lat           205
Lng           205
Max Temp      205
Humidity      205
Cloudiness    205
Wind Speed    205
dtype: int64

In [36]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Mayo,US,83.17,38.8876,-76.5119,
2,Guerrero Negro,MX,82.06,27.9769,-114.0611,
4,Dickinson,US,83.23,29.4608,-95.0513,
5,Alugan,PH,78.03,12.2188,125.4808,
6,Rikitea,PF,73.27,-23.1203,-134.9692,
7,Bambous Virieux,MU,72.21,-20.3428,57.7575,
11,Katsuura,JP,80.71,35.1333,140.3000,
16,Campoverde,PE,73.09,-8.4753,-74.8058,
18,Bethel,US,80.35,41.3712,-73.4140,
22,Wanning,CN,80.04,18.8003,110.3967,


Place Search API doc:
https://developers.google.com/maps/documentation/places/web-service/search#PlaceSearchRequests%0D%0A

In [38]:
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

In [39]:
# https://requests.kennethreitz.org/en/master/api/#main-interface

In [42]:
params = {
    "key": g_key,
    "type": "lodging",
    "radius": 5000
}

In [45]:
# Iterate through DataFrame
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key 
    params["location"] = f"{lat},{lng}"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"] 
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [46]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Mayo,US,83.17,38.8876,-76.5119,Resorts Resource Group International
2,Guerrero Negro,MX,82.06,27.9769,-114.0611,"Casa Laguna, Bed & Breakfast"
4,Dickinson,US,83.23,29.4608,-95.0513,Hampton Inn & Suites Houston/League City
5,Alugan,PH,78.03,12.2188,125.4808,JM Lodge
6,Rikitea,PF,73.27,-23.1203,-134.9692,People ThankYou
7,Bambous Virieux,MU,72.21,-20.3428,57.7575,Casa Tia Villa
11,Katsuura,JP,80.71,35.1333,140.3000,Katsuura Hotel Mikazuki
16,Campoverde,PE,73.09,-8.4753,-74.8058,Hospedaje Coral's
18,Bethel,US,80.35,41.3712,-73.4140,Courtyard by Marriott Danbury
22,Wanning,CN,80.04,18.8003,110.3967,Paradise Hotel


### Add a pop-up marker for each city

In [51]:
"""
Description List: <dl>
Description Term: <dt>
Description Definition: <dd>
Description Term: <dt>
Description Definition: <dd>
"""

'\nDescription List: <dl>\nDescription Term: <dt>\nDescription Definition: <dd>\nDescription Term: <dt>\nDescription Definition: <dd>\n'

In [57]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [58]:
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [59]:
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]

fig = create_heat_map(locations, max_temp, hotel_info)
fig

Figure(layout=FigureLayout(height='420px'))